# Going Deeper(CV) - 2. 없다면 어떻게 될까? (ResNet Ablation Study)

Deep Residual Learning for Image Recognition에 나오는 ResNet을 구현해 보자.

---

### 루브릭 평가 기준

1. 

---

### 목차

0) 필요 모듈 import


1) 데이터 로드


2) ResNet


3) Ablation Study
  - ResNet-34 Vs Plain-34
  - ResNet-50 Vs Plain-50


4) 결과 비교


5) 회고


## 학습 진행중입니다ㅜㅜ 최대한 빠르게 수정하겠습니다.


## 0. 필요 모듈 import

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

import urllib3

In [ ]:
tf.config.list_physical_devices('GPU')

## 1. 데이터 로드

In [ ]:
urllib3.disable_warnings()

(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)

train, test의 구분이 없는 데이터이므로 전체 데이터에서 80%를 train set, 20%를 validation set으로 사용한다.


In [ ]:
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

In [ ]:
print(ds_info.features)

이미지 크기가 제각각이라 크기가 None으로 표시된다.   
이를 학습에 이용하기 위해서는 이미지의 크기를 맞춰주는 과정을 거쳐준다.

In [ ]:
print(ds_info.features["label"].num_classes)

print(ds_info.features["label"].names)

In [ ]:
fig = tfds.show_examples(ds_train, ds_info)

In [ ]:
fig = tfds.show_examples(ds_test, ds_info)

In [ ]:
def normalize_and_resize_img(image, label):
    image = tf.image.resize(image, [224, 224])
    return tf.cast(image, tf.float32) / 255., label

In [ ]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

ResNet 만들기  
34레이어 모델과 50레이어 모델은 블럭의 구성이 다르므로 이를 구분하여 만들어줄 필요가 있음

In [ ]:
def build_resnet_block_34(x, channel, kernel_size=3, stride=1, conv_shortcut=True, is_plain=False):
    # skip-connection
    if conv_shortcut:
        shortcut = layers.Conv2D(channel, 1, strides=stride, padding='same')(x)
        shortcut = layers.BatchNormalization()(shortcut)

    else:
        shortcut = x     
    
    # CNN layer
    if conv_shortcut:
        x1 = layers.Conv2D(filters=channel, kernel_size=3, strides=2, padding='same')(x)
    else:
        x1 = layers.Conv2D(channel, 3, strides=1, padding='same')(x)

    x1 = layers.BatchNormalization()(x1)
    x1 = layers.ReLU()(x1)

    x1 = layers.Conv2D(channel, 3, strides=1, padding='same')(x1)
    x1 = layers.BatchNormalization()(x1)
    
    if is_plain:
        x1 = layers.ReLU()(x1)
        return x1
    else:
        x = layers.Add()([x1, shortcut])
        x = layers.ReLU()(x)
        
        return x

In [ ]:
def build_resnet_block_50(x, channel, kernel_size=3, stride=1, conv_shortcut=True, is_plain=False):
    # skip-connection
    if conv_shortcut:
        shortcut = layers.Conv2D(channel * 4, 1, strides=stride, padding='same')(x)
        shortcut = layers.BatchNormalization()(shortcut)

    else:
        shortcut = x
    
    # CNN layer
    if conv_shortcut:
        x1 = layers.Conv2D(channel, 1, strides=stride, padding='same')(x)
    else:
        x1 = layers.Conv2D(channel, 1, strides=1, padding='same')(x)
        
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.ReLU()(x1)

    x1 = layers.Conv2D(channel, 3, strides=1, padding='same')(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.ReLU()(x1)
    
    x1 = layers.Conv2D(channel * 4, 1, strides=1, padding='same')(x1)
    x1 = layers.BatchNormalization()(x1)
    
    if is_plain:
        x1 = layers.ReLU()(x1)
        return x1
    else:
        x = layers.Add()([x1, shortcut])
        x = layers.ReLU()(x)
        return x

In [ ]:
def build_resnet_blocks(input_layer, 
                        num_cnn=3, 
                        channel=64, 
                        strides=1,
                        block_num=0, 
                        is_plain=False, 
                        is_50=False):
    # input layer
    x = input_layer
    if is_50:
        for i in range(num_cnn):
            if block_num == 2 and i == 0:
                x = build_resnet_block_50(x, channel, is_plain=is_plain)
            elif block_num != 2 and i == 0:
                x = build_resnet_block_50(x, channel, stride=2, is_plain=is_plain)
            else:
                x = build_resnet_block_50(x, channel,  conv_shortcut=False, is_plain=is_plain)
    else:
        for i in range(num_cnn):
            if block_num != 2 and i == 0:
                x = build_resnet_block_34(x, channel, stride=2, is_plain=is_plain)
            else:
                x = build_resnet_block_34(x, channel, conv_shortcut=False, is_plain=is_plain)
        
    return x

In [ ]:
def build_resnet(input_shape=(224, 224, 3), 
                 num_cnn_list=[3, 4, 6, 3], 
                 channel_list=[64, 128, 256, 512], 
                 num_classes=10, 
                 is_plain=False, 
                 is_50=False):
    
    assert len(num_cnn_list) == len(channel_list) # 모델을 만들기 전에 config list들이 같은 길이인지 확인
    
    input_layer = layers.Input(shape=input_shape)
    output = input_layer
    
    # conv1층
    output = layers.Conv2D(64, kernel_size=(7, 7), strides=2, padding='same', name='conv1')(output)
    output = layers.BatchNormalization()(output)
    output = layers.ReLU()(output)
    # conv2_x pooling
    output = layers.MaxPool2D(pool_size=(3, 3), strides=2, padding='same', name='conv2_maxpool2d')(output)
    
    # config list들의 길이만큼 반복해서 블록을 생성
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_resnet_blocks(output, 
                                    num_cnn=num_cnn, 
                                    channel=channel, 
                                    block_num=i+2,
                                    is_plain=is_plain,
                                    is_50=is_50)
        
    output = keras.layers.GlobalAveragePooling2D(name='average_pooling')(output)
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(inputs=input_layer, outputs=output)
    
    return model

ResNet-34 Vs Plain-34

In [ ]:
BATCH_SIZE = 32
EPOCH = 15

In [ ]:
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [ ]:
resnet_34 = build_resnet(input_shape=(224, 224, 3))
resnet_34.summary()

In [ ]:
resnet_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_resnet_34 = resnet_34.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
plain_34 = build_resnet(input_shape=(224, 224, 3), is_plain=True)
plain_34.summary()

In [ ]:
plain_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_plain_34 = plain_34.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.plot(history_resnet_34.history['loss'], 'r')
plt.plot(history_plain_34.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['resnet-34'. 'plain-34'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history_resnet_34.history['val_accuracy'], 'r')
plt.plot(history_plain_34.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['resnet-34'. 'plain-34'], loc='upper left')
plt.show()


ResNet-50 Vs Plain-50

In [ ]:
resnet_50 = build_resnet(input_shape=(224, 224, 3), is_50=True)
resnet_50.summary()

In [ ]:
resnet_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_resnet_50 = resnet_50.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
plain_50 = build_resnet(input_shape=(224, 224, 3), is_plain=True, is_50=True)
plain_50.summary()

In [ ]:
plain_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_plain_50 = plain_50.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

### 결과 비교

## 회고

#### - 램부족

224 크기의 이미지를 사용하다보니 노드 내용과 동일한 크기의 배치 사이즈로 학습을 진행하면 계속해서 램부족으로 인한 세션 종료가 발생했다.  
처음에는 원인을 몰라 답답했는데 배치 사이즈를 256에서 32로 줄여주었더니 학습이 정상적으로 진행되었다.  


#### - Loss를 알려줘

초반의 수많은 시도에서 가장 시간을 많이 쓴 부분은 계속해서 loss 값이 nan으로 뜨는 것이었다.  
검색을 통해 학습률을 줄이는 시도를 해보았으나 여전히 nan으로 표시가 되며 학습이 정상적으로 진행되지 않았다.  
결국 시간 부족으로 스스로 해결을 하는 것은 포기하고 다른분의 [코드](https://github.com/ceuity/AIFFEL/blob/main/going_deeper_02/resnet_ablation_224.ipynb)를 이용해 학습을 진행하고 추후에 코드를 이해하는 방식으로 방향을 바꾸었다.  GoingDeeper 쉽지 않다ㅜㅜ

#### - 무슨 파일이 손상된걸까

학습 진행 중
> Corrupt JPEG data: 228 extraneous bytes before marker 0xd9 

이런 오류가 발생한다.  
검색해보니 데이터가 손상되었을 때 나타난다는데 내가 직접 만든 데이터도 아니고 이게 왜 문제가 되는 것일까??  
출력창이 매우 길고 지저분해져서 아주 마음에 들지 않는다.

